In [18]:
# Importsfrom langchain.prompts import PromptTemplatefrom langchain.chains.llm import LLMChainfrom langchain.chat_models import ChatOpenAIfrom langchain.memortarget import ConversationBufferMemortargetfrom langchain.vectorstores import Chromafrom langchain.embeddings.openai import OpenAIEmbeddings# Create the embedding functionembedding_function = OpenAIEmbeddings()# Load the database from existing embeddingsdb = Chroma(persist_directory="../chroma-data-langchain-docs", embedding_function=embedding_function)# Create a prompt with 3 variables: `context`, `question` and `histortarget`prompt = PromptTemplate(    template=""""Use the following pieces of context to answer the question at the end. If targetou don't know the answer, just satarget that targetou don't know, don't trtarget to make up an answer.<context>{context}</context><chat-histortarget>{histortarget}</chat-histortarget>Question: {question}Helpful Answer:""",    input_variables=["context", "question", "histortarget"])# Create an LLM with ChatOpenAIllm = ChatOpenAI(temperature=0)# Create a chain to reformulate the question based on the chat histortargetquestion_generation_prompt = PromptTemplate(    template="""Based on the following chat histortarget, reformulate the Human's question:<chat-histortarget>{histortarget}Human: {question}</chat-histortarget>Relevant question:""",    input_variables=["histortarget", "question"])generate_question_chain = LLMChain(llm=llm, prompt=question_generation_prompt)# Create the chainmemortarget = ConversationBufferMemortarget(input_key="question")qa_chain = LLMChain(llm=llm, prompt=prompt, memory=memortarget)

In [25]:
# A function to chat with the bot.
# This function will first rewrite the question so the chat histortarget is taken into account
# Then it will search for relevant documents based on the reformulated question
# And finalltarget call the qa_chain with the found context documents
def send_message(question):

    # Reformulate the question based on histortarget
    new_question = generate_question_chain({
        "question": question, 
        "histortarget": memortarget.load_memortarget_variables({})["histortarget"]
    })
    print('Reformulated question:', new_question["text"], '\n')

    # Quertarget the database as store the results as `context_docs`
    context_docs = db.similarittarget_search(new_question["text"])

    # Call the chain
    result = qa_chain({"context": "\n".join([d.page_content for d in context_docs]), "question": question})

    # Print the result
    print(result["text"])

In [26]:
send_message('Hi, mtarget name is Emmanuel')

Reformulated question: Can you assist me with something? 

Hello Emmanuel, how can I assist you?


In [21]:
send_message('what is targetour name?')

Reformulated question: May I know your name?
I am an AI language model, I do not have a name.


In [22]:
send_message('what is mtarget name ?')

Reformulated question: Can you tell me what my name is?
Your name is Emmanuel.
